In [1]:
import os 
import json
from django.contrib.gis.geos import GEOSGeometry
import time 
import shutil 

import numpy as np
from shapely.geometry import mapping
import pandas as pd
import fiona
import geopandas as gpd
from fiona.crs import to_string
from fiona.crs import from_epsg
from shapely.geometry import shape, Point, Polygon
from fiona.crs import to_string

from madmex.overlay.conversions import train_object_to_feature
from madmex.util.spatial import geometry_transform
from madmex.models import TrainClassification

Checar primero:

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/agriculture_madmex_app/3a_assign_labels_and_insert_in_db_for_agriculture.ipynb

# 7) Random sample para cada categoría generada de Kmeans

**La muestra debe generarse de modo que incluya los polígonos dados por inecol más otros y que en total sean 1000**

Intersección y contención del geopandas dataframe con el shapefile:

In [280]:
input_file2='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/02-11-2020_clusters_40_-31_nclusters_7_lcc.shp'

In [281]:
gdf_df_clusters = gpd.read_file(input_file2)

In [282]:
gdf_df_clusters.head()

,features_i,preds,geometry
0,537885,4,"POLYGON ((2978161.815 920672.919, 2978181.824 ..."
1,537675,4,"POLYGON ((2978451.936 920672.974, 2978622.007 ..."
2,537799,4,"POLYGON ((2978642.015 920673.010, 2978712.044 ..."
3,537676,4,"POLYGON ((2978812.086 920673.042, 2978852.103 ..."
4,537886,4,"POLYGON ((2979522.381 920673.178, 2979542.390 ..."


In [283]:
len(gdf_df_clusters.index)

538023

In [284]:
crs = {'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [287]:
crs = gdf_df_clusters.crs

In [288]:
nclusters=len(gdf_df_clusters['preds'].unique())

**Leer puntos de cultivos ya registrados con su valor numérico en la tabla TrainClassification**

Obtener extent en coordenadas geográficas de polígonos clúster

https://gis.stackexchange.com/questions/266730/filter-by-bounding-box-in-geopandas

http://geopandas.org/set_operations.html

In [289]:
bbox = gdf_df_clusters.total_bounds

In [290]:
p1 = Point(bbox[0], bbox[3])
p2 = Point(bbox[2], bbox[3])
p3 = Point(bbox[2], bbox[1])
p4 = Point(bbox[0], bbox[1])
p1 = shape(geometry_transform(mapping(p1),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
p2 = shape(geometry_transform(mapping(p2),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
p3 = shape(geometry_transform(mapping(p3),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
p4 = shape(geometry_transform(mapping(p4),crs_out="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",
                              crs_in=crs))
np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

In [291]:
bb_polygon = Polygon([np1, np2, np3, np4])

In [292]:
bb_polygon_for_query = GEOSGeometry(json.dumps(mapping(bb_polygon)))

In [293]:
bb_polygon_for_query

In [294]:
training_set = 'cultivos2019_inecol'

In [295]:
query_set = TrainClassification.objects.filter(train_object__the_geom__intersects=bb_polygon_for_query,
                                               training_set=training_set).prefetch_related('train_object', 'interpret_tag')

In [296]:
len(query_set)

1358

In [297]:
to_string(crs)

'+datum=WGS84 +lat_0=12 +lat_1=17.5 +lat_2=29.5 +lon_0=-102 +no_defs +proj=lcc +units=m +x_0=2500000 +y_0=0'

In [298]:
fc_cultivos = [train_object_to_feature(x, to_string(crs)) for x in query_set]

In [299]:
fc_cultivos[0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [2996020.510094165, 871031.2405095949]},
 'properties': {'class': 16}}

In [300]:
# Build the GeoDataFrame from Fiona Collection
gdf_poi = gpd.GeoDataFrame.from_features([feature for feature in fc_cultivos], crs=crs)


In [301]:
gdf_poi.head()

,geometry,class
0,POINT (2996020.510 871031.241),16
1,POINT (2996171.754 872139.508),16
2,POINT (2995121.887 873212.291),16
3,POINT (2994861.211 873668.657),16
4,POINT (2994967.629 873496.546),16


In [302]:
len(gdf_poi.index)

1358

In [303]:
gdf_poi.head()

,geometry,class
0,POINT (2996020.510 871031.241),16
1,POINT (2996171.754 872139.508),16
2,POINT (2995121.887 873212.291),16
3,POINT (2994861.211 873668.657),16
4,POINT (2994967.629 873496.546),16


**Interesan los id's que intersectan los puntos pues se extraerán tales id's del gdf_df_clusters**

Obtener `features_id` de `gdf_df_clusters`  que intersectan `gdf_poi`

In [308]:
gdf_df_clusters_poi_intersects = gpd.sjoin(gdf_df_clusters.reset_index(drop=True),
                                           gdf_poi.reset_index(drop=True),
                                           op='intersects',
                                           how='inner')

In [309]:
gdf_df_clusters_poi_intersects.head()

,features_i,preds,geometry,index_right,class
2296,535938,1,"POLYGON ((3021949.993 920671.680, 3021970.002 ...",264,20
4343,533861,1,"POLYGON ((3014827.009 920511.088, 3014837.013 ...",259,20
4533,533389,1,"POLYGON ((3013646.497 920401.448, 3013656.502 ...",255,20
6898,531284,1,"POLYGON ((3017187.914 920153.543, 3017207.923 ...",652,20
7438,530710,1,"POLYGON ((3013766.535 920341.801, 3013826.560 ...",254,20


In [310]:
len(gdf_df_clusters_poi_intersects.index)

1358

In [311]:
gdf_df_clusters_keep = gpd.GeoDataFrame()

In [312]:
gdf_df_clusters_keep['features_i'] = gdf_df_clusters_poi_intersects['features_i']
gdf_df_clusters_keep['preds'] = gdf_df_clusters_poi_intersects['preds']
gdf_df_clusters_keep['geometry'] = gdf_df_clusters_poi_intersects['geometry']
gdf_df_clusters_keep['class'] = gdf_df_clusters_poi_intersects['class']

In [313]:
gdf_df_clusters_keep.head()

,features_i,preds,geometry,class
2296,535938,1,"POLYGON ((3021949.993 920671.680, 3021970.002 ...",20
4343,533861,1,"POLYGON ((3014827.009 920511.088, 3014837.013 ...",20
4533,533389,1,"POLYGON ((3013646.497 920401.448, 3013656.502 ...",20
6898,531284,1,"POLYGON ((3017187.914 920153.543, 3017207.923 ...",20
7438,530710,1,"POLYGON ((3013766.535 920341.801, 3013826.560 ...",20


In [314]:
len(gdf_df_clusters_keep.index)

1358

In [315]:
gdf_df_clusters_keep.crs = crs

In [316]:
gdf_df_clusters_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

**If number of rows are less than 1000 then filter_out rows that don't intersect points and then sample**

In [317]:
t_keep_rows = tuple(gdf_df_clusters_keep['features_i'])

In [318]:
len(t_keep_rows)

1358

In [319]:
t_keep_rows[0:5]

(535938, 533861, 533389, 531284, 530710)

In [320]:
gdf_df_clusters_to_sample = gdf_df_clusters.loc[~gdf_df_clusters['features_i'].isin(t_keep_rows)]

In [321]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,537885,4,"POLYGON ((2978161.815 920672.919, 2978181.824 ..."
1,537675,4,"POLYGON ((2978451.936 920672.974, 2978622.007 ..."
2,537799,4,"POLYGON ((2978642.015 920673.010, 2978712.044 ..."
3,537676,4,"POLYGON ((2978812.086 920673.042, 2978852.103 ..."
4,537886,4,"POLYGON ((2979522.381 920673.178, 2979542.390 ..."


In [322]:
len(gdf_df_clusters_to_sample.index)

536678

In [323]:
len(gdf_df_clusters_keep.index)+len(gdf_df_clusters_to_sample.index)

538036

In [324]:
len(gdf_df_clusters.index)

538023

**Extraer muestra**

Using https://stackoverflow.com/questions/41345289/getting-a-random-sample-in-python-dataframe-by-category

In [325]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,537885,4,"POLYGON ((2978161.815 920672.919, 2978181.824 ..."
1,537675,4,"POLYGON ((2978451.936 920672.974, 2978622.007 ..."
2,537799,4,"POLYGON ((2978642.015 920673.010, 2978712.044 ..."
3,537676,4,"POLYGON ((2978812.086 920673.042, 2978852.103 ..."
4,537886,4,"POLYGON ((2979522.381 920673.178, 2979542.390 ..."


In [326]:
nmultipolygons=len(gdf_df_clusters_to_sample.index)

In [327]:
nmultipolygons

536678

In [328]:
total_mpolygons = 1000
if total_mpolygons>len(t_keep_rows):
    nsample=total_mpolygons-len(t_keep_rows)
else:
    nsample=total_mpolygons

In [329]:
nsample

1000

In [330]:
nclusters

7

In [331]:
if nmultipolygons>nsample:
    nsample_per_cluster=int(nsample/nclusters)+1
    nsample=nsample_per_cluster*nclusters
else:
    nsample_per_cluster=int(nmultipolygons/nclusters)+1
    nsample=nsample_per_cluster*nclusters
 

In [332]:
(nmultipolygons,nsample_per_cluster,nsample_per_cluster*nclusters,nsample)

(536678, 143, 1001, 1001)

In [333]:
gdf_df_clusters_sample=gdf_df_clusters_to_sample.groupby('preds').apply(lambda s: s.sample(min(len(s),
                                                                                               nsample_per_cluster),
                                                                                          random_state=1))

In [334]:
gdf_df_clusters_sample.head()

features_i  preds  \
preds                             
0     29376       508418      0   
      394762      143214      0   
      42576       495149      0   
      430958      107088      0   
      442542       95690      0   

                                                       geometry  
preds                                                            
0     29376   POLYGON ((2982162.999 918147.588, 2982173.004 ...  
      394762  POLYGON ((3001684.530 885014.467, 3001714.542 ...  
      42576   POLYGON ((3007783.403 917088.492, 3007803.411 ...  
      430958  POLYGON ((2987237.993 881172.905, 2987268.005 ...  
      442542  POLYGON ((3007055.667 879943.670, 3007075.675 ...

In [335]:
len(gdf_df_clusters_sample.index)

1001

In [336]:
gdf_df_clusters_sample['preds'].value_counts()

6    143
5    143
4    143
3    143
2    143
1    143
0    143
Name: preds, dtype: int64

In [337]:
gdf_df_clusters_sample.reset_index(drop=True,inplace=True)

In [338]:
gdf_df_clusters_sample.head()

,features_i,preds,geometry
0,508418,0,"POLYGON ((2982162.999 918147.588, 2982173.004 ..."
1,143214,0,"POLYGON ((3001684.530 885014.467, 3001714.542 ..."
2,495149,0,"POLYGON ((3007783.403 917088.492, 3007803.411 ..."
3,107088,0,"POLYGON ((2987237.993 881172.905, 2987268.005 ..."
4,95690,0,"POLYGON ((3007055.667 879943.670, 3007075.675 ..."


In [339]:
len(gdf_df_clusters_sample.index)

1001

**Union of sample and keep polygons**

In [340]:
gdf_sampled_keep = pd.concat([gdf_df_clusters_sample,gdf_df_clusters_keep],
                             ignore_index=True,sort=False).pipe(gpd.GeoDataFrame)

In [341]:
gdf_sampled_keep.head()

,features_i,preds,geometry,class
0,508418,0,"POLYGON ((2982162.999 918147.588, 2982173.004 ...",NaN
1,143214,0,"POLYGON ((3001684.530 885014.467, 3001714.542 ...",NaN
2,495149,0,"POLYGON ((3007783.403 917088.492, 3007803.411 ...",NaN
3,107088,0,"POLYGON ((2987237.993 881172.905, 2987268.005 ...",NaN
4,95690,0,"POLYGON ((3007055.667 879943.670, 3007075.675 ...",NaN


In [342]:
len(gdf_sampled_keep.index)

2359

In [343]:
gdf_sampled_keep['class'].isna()

0        True
1        True
2        True
3        True
4        True
        ...  
2354    False
2355    False
2356    False
2357    False
2358    False
Name: class, Length: 2359, dtype: bool

In [344]:
if sum(gdf_sampled_keep['class'].isna())>0:
    gdf_sampled_keep.loc[gdf_sampled_keep['class'].isna(),'class'] = ''

In [345]:
sum(gdf_sampled_keep['class'].isna())

0

In [346]:
gdf_sampled_keep.head()

,features_i,preds,geometry,class
0,508418,0,"POLYGON ((2982162.999 918147.588, 2982173.004 ...",
1,143214,0,"POLYGON ((3001684.530 885014.467, 3001714.542 ...",
2,495149,0,"POLYGON ((3007783.403 917088.492, 3007803.411 ...",
3,107088,0,"POLYGON ((2987237.993 881172.905, 2987268.005 ...",
4,95690,0,"POLYGON ((3007055.667 879943.670, 3007075.675 ...",


In [347]:
len(gdf_sampled_keep.index)

2359

# 8) Write results to file

In [348]:
date_today = '02-11-2020'
dc_tile = '40_-31'
path_result_dc_tile = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/'
features_string = 'ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled'
total_mpolygons = len(gdf_sampled_keep.index)

**Geopackage**

In [349]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [350]:
path_results_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/'

In [351]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [352]:
path_result_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/'

In [353]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [354]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class.gpkg'

In [355]:
clusters_filename

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/02-11-2020_clusters_40_-31_nclusters_7_sample_2359_with_class.gpkg'

In [356]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class'

In [357]:
layer_name

'02-11-2020_clusters_40_-31_nclusters_7_sample_2359_with_class'

In [358]:
gdf_sampled_keep.crs

In [359]:
gdf_sampled_keep.crs = crs

In [360]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [361]:
start_time=time.time()

gdf_sampled_keep.to_file(clusters_filename,
                         layer=layer_name,
                         driver='GPKG')
end_time=time.time()

In [362]:
secs = end_time-start_time
secs

0.9517974853515625

In [363]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/02-11-2020_clusters_40_-31_nclusters_7_sample_2359_with_class.gpkg'

**Shapefile**

In [364]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '_with_class.shp'

In [365]:
clusters_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/02-11-2020_clusters_40_-31_nclusters_7_sample_2359_with_class.shp'

In [366]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [367]:
gdf_sampled_keep.crs = crs

In [368]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [369]:
start_time=time.time()
gdf_sampled_keep.to_file(clusters_filename)
end_time=time.time()

In [370]:
secs = end_time-start_time
secs

1.1457288265228271

# 12) Registrar segmentos con etiquetas automáticas en trainclassificationlabeledbyapp con esquema "automatic_7_classes"

**Command if shapefile of reference do have labels:**

In [371]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector \
/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/02-11-2020_clusters_40_-31_nclusters_7_sample_2359_with_class.shp \
--scheme automatic_7_classes --year 2020 --name agricultura_7_classes_2359_with_class_added \
--field preds --app --train_interpreted --scheme_interpreted agricultura_cultivos2019 --field_interpreted class --dc_tile 40_-31

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean_resampled-ndvi_max_resampled-ndvi_min_resampled/02-11-2020/40_-31/02-11-2020_clusters_40_-31_nclusters_7_sample_2359_with_class.shp', 'scheme': 'automatic_7_classes', 'field': 'preds', 'name': 'agricultura_7_classes_2359_with_class_added', 'year': '2020', 'app': True, 'train_interpreted': True, 'scheme_interpreted': 'agricultura_cultivos2019', 'field_interpreted': 'class', 'dc_tile': '40_-31'}
Command execution is done in 85.32834887504578 seconds.
2020-02-13 18:14:37,252 - madmex.management.base - base execute: Command execution is done in 85.32834887504578 seconds.


**Command if shapefile of reference don't have labels:**

In [81]:
#%%bash
#/home/madmex_user/.local/bin/antares ingest_training_from_vector \
#/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp \
#--scheme automatic_4_classes --year 2020 --name agricultura_4_clases_sample_1000_with_class --field preds --app

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000_with_class.shp', 'scheme': 'automatic_4_classes', 'field': 'preds', 'name': 'agricultura_4_clases_sample_1000_with_class', 'year': '2020', 'app': True}
Command execution is done in 13.483619928359985 seconds.
2020-02-06 20:55:45,824 - madmex.management.base - base execute: Command execution is done in 13.483619928359985 seconds.
